In [2]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

In [3]:
!wget https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
!unzip android_figurine.zip
train_dataset_path = "android_figurine/train"
validation_dataset_path = "android_figurine/validation"


--2024-09-06 17:31:55--  https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.214.27, 142.250.179.219, 142.251.36.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.214.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28607322 (27M) [application/zip]
Saving to: ‘android_figurine.zip’

android_figurine.zi 100%[===================>]  27,28M  5,26MB/s    in 6,8s    

2024-09-06 17:32:03 (4,03 MB/s) - ‘android_figurine.zip’ saved [28607322/28607322]

Archive:  android_figurine.zip
   creating: android_figurine/
   creating: android_figurine/validation/
  inflating: android_figurine/validation/labels.json  
   creating: android_figurine/validation/images/
  inflating: android_figurine/validation/images/IMG_0502.jpg  
  inflating: android_figurine/validation/images/IMG_0497.jpg  
  inflating: android_figurine/validation/images/IMG_0501.jpg  

In [4]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")


0: background
1: android
2: pig_android


In [5]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)


INFO:tensorflow:On image 0


2024-09-06 17:46:20.708195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-06 17:46:20.996679: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:On image 0
train_data size:  62
validation_data size:  10


In [6]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [7]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)


/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

INFO:tensorflow:Training the models...


Epoch 1/30


/home/jesse/.local/lib/python3.8/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


7/7 [==============================] - 42s 2s/step - total_loss: 5.9634 - cls_loss: 5.7791 - box_loss: 0.0024 - model_loss: 5.9005 - val_total_loss: 1.3662 - val_cls_loss: 1.2293 - val_box_loss: 0.0015 - val_model_loss: 1.3034
Epoch 2/30
7/7 [==============================] - 9s 1s/step - total_loss: 1.3948 - cls_loss: 1.2035 - box_loss: 0.0026 - model_loss: 1.3320 - val_total_loss: 1.2406 - val_cls_loss: 1.1423 - val_box_loss: 7.0817e-04 - val_model_loss: 1.1777
Epoch 3/30
7/7 [==============================] - 9s 1s/step - total_loss: 1.2366 - cls_loss: 1.0967 - box_loss: 0.0015 - model_loss: 1.1737 - val_total_loss: 1.1167 - val_cls_loss: 1.0246 - val_box_loss: 5.8479e-04 - val_model_loss: 1.0538
Epoch 4/30
7/7 [==============================] - 10s 1s/step - total_loss: 0.9968 - cls_loss: 0.8808 - box_loss: 0.0011 - model_loss: 0.9339 - val_total_loss: 0.9243 - val_cls_loss: 0.8260 - val_box_loss: 7.0774e-04 - val_model_loss: 0.8614
Epoch 5/30
7/7 [==============================] -

In [8]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


3/3 [==============================] - 1s 118ms/step - total_loss: 0.2345 - cls_loss: 0.1558 - box_loss: 3.1640e-04 - model_loss: 0.1717
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.902
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.902
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.914
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.914
 Average Recall     (AR)

In [9]:
model.export_model()

Exporting a floating point model


/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmpuavimmrc/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpuavimmrc/saved_model/assets
2024-09-06 17:56:04.817997: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-06 17:56:04.818093: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-06 17:56:04.818384: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpuavimmrc/saved_model
2024-09-06 17:56:04.896055: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-06 17:56:04.896084: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpuavimmrc/saved_model
2024-09-06 17:56:05.036500: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-09-06 17:56:05.094855: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-09-06 17:56:05.987561: I tensorflow/cc/saved_model/

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model.tflite


In [10]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")
# Quantization seems to significiantly reduce results! Skip quantization for the real model

/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Epoch 1/10


/home/jesse/.local/lib/python3.8/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


15/15 [==============================] - 65s 1s/step - total_loss: 2.7510 - cls_loss: 1.1556 - box_loss: 0.0319 - model_loss: 2.7510 - val_total_loss: 2.6202 - val_cls_loss: 1.4536 - val_box_loss: 0.0233 - val_model_loss: 2.6202
Epoch 2/10
15/15 [==============================] - 12s 831ms/step - total_loss: 1.8082 - cls_loss: 0.9480 - box_loss: 0.0172 - model_loss: 1.8082 - val_total_loss: 2.4070 - val_cls_loss: 1.4715 - val_box_loss: 0.0187 - val_model_loss: 2.4070
Epoch 3/10
15/15 [==============================] - 13s 851ms/step - total_loss: 1.3004 - cls_loss: 0.7828 - box_loss: 0.0104 - model_loss: 1.3004 - val_total_loss: 2.3420 - val_cls_loss: 1.4476 - val_box_loss: 0.0179 - val_model_loss: 2.3420
Epoch 4/10
15/15 [==============================] - 12s 829ms/step - total_loss: 1.1620 - cls_loss: 0.7007 - box_loss: 0.0092 - model_loss: 1.1620 - val_total_loss: 1.9933 - val_cls_loss: 1.2994 - val_box_loss: 0.0139 - val_model_loss: 1.9933
Epoch 5/10
15/15 [========================

In [11]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_2 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

15/15 [==============================] - 64s 1s/step - total_loss: 2.6565 - cls_loss: 1.1689 - box_loss: 0.0298 - model_loss: 2.6565 - val_total_loss: 1.8481 - val_cls_loss: 1.1372 - val_box_loss: 0.0142 - val_model_loss: 1.8481
Epoch 2/15
15/15 [==============================] - 12s 821ms/step - total_loss: 2.0396 - cls_loss: 1.0502 - box_loss: 0.0198 - model_loss: 2.0396 - val_total_loss: 1.5235 - val_cls_loss: 0.9923 - val_box_loss: 0.0106 - val_model_loss: 1.5235
Epoch 3/15
15/15 [==============================] - 12s 791ms/step - total_loss: 1.7346 - cls_loss: 0.9916 - box_loss: 0.0149 - model_loss: 1.7346 - val_total_loss: 1.4391 - val_cls_loss: 0.9336 - val_box_loss: 0.0101 - val_model_loss: 1.4391
Epoch 4/15
15/15 [==============================] - 12s 822ms/step - total_loss: 1.5603 - cls_loss: 0.9261 - box_loss: 0.0127 - model_loss: 1.5603 - val_total_loss: 1.4560 - val_cls_loss: 0.9483 - val_box_loss: 0.0102 - val_model_loss: 1.4560
Epoch 5/15
15/15 [========================

In [12]:
model.export_model('model_int8_qat.tflite')

Exporting a qat int8 model


/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmp47g6p_8q/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp47g6p_8q/saved_model/assets
2024-09-06 18:06:38.124848: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-06 18:06:38.124872: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-06 18:06:38.125055: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp47g6p_8q/saved_model
2024-09-06 18:06:38.228674: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-06 18:06:38.228708: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp47g6p_8q/saved_model
2024-09-06 18:06:38.538127: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-09-06 18:06:40.583126: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp47g6p_8q/saved_model
2024-09-06 18:06:41.327456: I ten

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_int8_qat.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_int8_qat.tflite


In [13]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_float16()
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)

/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_4 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_4 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmp2ldrkn9d/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp2ldrkn9d/saved_model/assets
2024-09-06 18:07:43.520887: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-09-06 18:07:43.520913: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-09-06 18:07:43.521097: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2ldrkn9d/saved_model
2024-09-06 18:07:43.583415: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-09-06 18:07:43.583445: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp2ldrkn9d/saved_model
2024-09-06 18:07:43.747602: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-09-06 18:07:44.622729: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp2ldrkn9d/saved_model
2024-09-06 18:07:45.043767: I ten

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_fp16.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_fp16.tflite


In [14]:
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
# The post-training quantization step seems to have no effect on the model evaluation, but it should make the model faster so ALWAYS include this step!

/home/jesse/.local/lib/python3.8/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


10/10 [==============================] - 5s 75ms/step - total_loss: 0.2221 - cls_loss: 0.1432 - box_loss: 3.1918e-04 - model_loss: 0.1592
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.902
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.902
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.914
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.914
 Average Recall     (AR